In [1]:
from keras.layers import Input, Dense, Lambda, Flatten, Activation, Merge, Concatenate, Add
from keras import layers
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler

import numpy as np
from scipy.stats import sem
from scipy.spatial.distance import squareform, pdist
from keras import backend as K
from keras.models import load_model
import tensorflow as tf


from models import vgg
from models.cvae import CVAE
from models.cvae_mod import CVAE as CVAE_mod
from utils.losses import von_mises_log_likelihood_np
from utils.angles import deg2bit, bit2deg
from utils.losses import mad_loss_tf, cosine_loss_tf, von_mises_loss_tf, maad_from_deg
from utils.losses import gaussian_kl_divergence_tf, gaussian_kl_divergence_np
from utils.losses import gaussian_log_likelihood_np, gaussian_log_likelihood_scipy, gaussian_log_likelihood_tf
from utils.losses  import von_mises_log_likelihood_tf, von_mises_log_likelihood_np
from utils.towncentre import load_towncentre
from utils.experiements import get_experiment_id

Using TensorFlow backend.


In [2]:
xtr, ytr_deg, xval, yval_deg, xte, yte_deg = load_towncentre('data/TownCentre.pkl.gz', canonical_split=True, verbose=1)

image_height, image_width = xtr.shape[1], xtr.shape[2]

xtr = np.tile(xtr, [5,1,1,1])

ytr_bit = deg2bit(ytr_deg)
yval_bit = deg2bit(yval_deg)
yte_bit = deg2bit(yte_deg)
yte_rad = np.deg2rad(yte_deg)

image_height, image_width, n_channels = xtr.shape[1:]
flatten_x_shape = xtr[0].flatten().shape[0]
phi_shape = yte_bit.shape[1]

Number of train samples: 6916
Number of validation samples: 874
Number of test samples: 904


In [36]:
def aug_data(x, y_deg, n_times=4, randomize_labels=True):
    x_aug = np.tile(x, [n_times, 1, 1, 1])
    y_deg_aug = np.tile(y_deg, [n_times])
    if randomize_labels:
        y_deg_aug = np.random.randint(0, 359, y_deg_aug.shape[0]).astype('float')
        y_deg_aug[0:y_deg.shape[0]] = y_deg
    return x_aug, y_deg_aug

In [37]:
xtr, ytr_deg = aug_data(xtr, ytr_deg)
xval, yval_deg = aug_data(xval, yval_deg)
xte, yte_deg = aug_data(xval, yval_deg)

In [39]:
ytr_deg

array([ 210.,   32.,  165., ...,  193.,  186.,  179.])